In [1]:
import pandas as pd
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from scipy.optimize import minimize
import statsmodels.api as sm
import numpy as np 
from sklearn.model_selection import train_test_split as tts, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error as mse

In [51]:
!pip install --upgrade --q statsmodels
import statsmodels.api as sm

### Import the provided dataset “AirI.csv.” The first column represents the response variable, and the rest are the predictors (input features). 

In [2]:
data = pd.read_csv('/Users/rebeccawagner/Documents/GitHub/Data 441/Data/AirI.csv',names=[i for i in range(-1,511)])
data.shape

(111, 512)

In [3]:
data.head()

,-1,0,1,2,3,4,5,6,7,8,...,501,502,503,504,505,506,507,508,509,510
0,3.4482,0.001314,0.15747,0.688300,0.152920,0.00000,0.00000,0.096603,0.52135,0.38205,...,-2.94590,0.219960,-0.69045,0.24044,-0.258640,1.33870,0.114820,-0.43046,0.82309,1.54350
1,3.3019,0.188490,0.55319,0.258290,0.000031,0.00000,0.00000,0.039008,0.43211,0.52716,...,-0.22680,-0.428570,0.74992,-0.11192,1.583000,0.88565,-0.053227,0.67796,-1.72010,-0.55051
2,2.2894,0.052169,0.43852,0.494030,0.015282,0.00000,0.00000,0.000000,0.00000,0.39484,...,1.15790,0.010118,0.73593,1.35600,0.057821,-1.44140,1.581800,1.25420,1.02540,-1.01890
3,2.6207,0.000000,0.00000,0.004213,0.109340,0.49344,0.39300,0.000000,0.00000,0.57854,...,-0.69767,1.577800,0.54899,-0.78042,1.136700,-0.84240,0.993120,-0.44576,-0.78778,-1.49090
4,2.8439,0.000000,0.00000,0.019504,0.260470,0.54986,0.17016,0.010568,0.30520,0.67046,...,-1.49170,-0.688640,-0.69924,0.36074,-0.685860,-0.76161,-0.354560,0.83093,-0.27100,0.38861


In [33]:
y = data[-1].values
x = data.loc[:,0:511].values

### Implement with a SKLearn wrapper the Square Root Lasso and Smoothly Clipped Absolute Deviations algorithms.

In [5]:
class SQRTLasso:
    def __init__(self, alpha = 0.5,intercept=True):
        self.alpha = alpha
        self.intercept = intercept
    
    def fit(self, x, y):
        alpha = self.alpha
        intercept = self.intercept
        self.xtrain_ = x
        self.ytrain_ = y
        self.fitted_model_ = sm.OLS(y,x).fit_regularized(method='sqrt_lasso', alpha=alpha)
        self.coef_ = self.fitted_model_.params
    
    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.ytrain_
        alpha = self.alpha
        intercept = self.intercept
        fitted_model = self.fitted_model_
        return  fitted_model.predict(x_new)

    def get_params(self, deep=True):
    # suppose this estimator has parameters "f", "iter" and "intercept"
        return {"alpha": self.alpha,"intercept":self.intercept}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

In [32]:
data_2 = pd.read_csv('/Users/rebeccawagner/Documents/GitHub/Data 441/Data/concrete.csv')
data_2

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [50]:
y = data['strength'].values
x = data[['cement','slag','ash','water','superplastic','coarseagg','fineagg','age']]

KeyError: 'strength'

In [45]:
def scad_penalty(beta_hat, lambda_val, a_val):
    is_linear = (np.abs(beta_hat) <= lambda_val)
    is_quadratic = np.logical_and(lambda_val < np.abs(beta_hat), np.abs(beta_hat) <= a_val * lambda_val)
    is_constant = (a_val * lambda_val) < np.abs(beta_hat)
    
    linear_part = lambda_val * np.abs(beta_hat) * is_linear
    quadratic_part = (2 * a_val * lambda_val * np.abs(beta_hat) - beta_hat**2 - lambda_val**2) / (2 * (a_val - 1)) * is_quadratic
    constant_part = (lambda_val**2 * (a_val + 1)) / 2 * is_constant
    return linear_part + quadratic_part + constant_part
    
def scad_derivative(beta_hat, lambda_val, a_val):
    return lambda_val * ((beta_hat <= lambda_val) + (a_val * lambda_val - beta_hat)*((a_val * lambda_val - beta_hat) > 0) / ((a_val - 1) * lambda_val) * (beta_hat > lambda_val))

def scad(beta, lam, a):
  beta = beta.flatten()
  beta = beta.reshape(-1,1)
  n = len(y)
  return 1/n*np.sum((y-x.dot(beta))**2) + np.sum(scad_penalty(beta,lam,a))

def dscad(beta, lam, a):
  beta = beta.flatten()
  beta = beta.reshape(-1,1)
  n = len(y)
  return np.array(-2/n*np.transpose(x).dot(y-x.dot(beta))+scad_derivative(beta,lam,a)).flatten()

In [46]:
class SCAD:
    
    def __init__(self, alpha, lam):
        self.alpha = alpha
        self.lam = lam

    def fit(self, x, y):

        alpha = self.alpha
        lam = self.lam 
        
        beta_0 = np.random.uniform(size=x.shape[1]).reshape(-1,1)
        output = minimize(scad, (beta_0, alpha, lam), method='L-BFGS-B', jac=dscad, options={'gtol': 1e-8, 'maxiter': 50000,'maxls': 25,'disp': True})
        self.coef_ = output.x

    def predict(self, x_new):
        check_is_fitted(self)
        beta_hat = self.coef_
        y_hat = beta_hat.dot(x_new)
        return y_hat

### Test Lasso & SCAD

In [47]:
xtrain, xtest, ytrain, ytest = tts(x, y, test_size = .3)

In [49]:
beta_0 = np.random.uniform(size=xtrain.shape[1]).reshape(-1,1)
alpha = 1
lam = 2

output = minimize(scad, (beta_0, alpha, lam), method='L-BFGS-B', jac=dscad, options={'gtol': 1e-8, 'maxiter': 50000,'maxls': 25,'disp': True})

/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/scipy/optimize/_minimize.py:515: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x0 = np.atleast_1d(np.asarray(x0))


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [48]:
scad = SCAD(alpha = 1, lam = 2)
scad.fit(xtrain,ytrain)

/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/scipy/optimize/_minimize.py:515: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x0 = np.atleast_1d(np.asarray(x0))


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [52]:
lasso = SQRTLasso()
lasso.fit(xtrain,ytrain)

ValueError: sqrt_lasso fitting requires the cvxopt module

### Determine the best choice of hyperparameters for the two methods (within finite ranges) by using GridsearchCV

In [104]:
params = {'alpha': [i for i in range(1,100)]}

gs_lasso = GridSearchCV(SQRTLasso(),
                      param_grid=params,
                      scoring='neg_mean_squared_error',
                      cv=5)

In [108]:
gs_lasso.fit(x,y)

ValueError: 
All the 495 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
495 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/statsmodels/regression/linear_model.py", line 1126, in _sqrt_lasso
    import cvxopt
ModuleNotFoundError: No module named 'cvxopt'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/var/folders/25/yq9cvx4n7714ymc6mt488mw80000gn/T/ipykernel_18239/1216489396.py", line 11, in fit
    self.fitted_mod

### Do the same with ElasticNet

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet

In [114]:
params = [{'alpha': [i for i in range(1,100)],
           'l1_ratio': [i for i in range(0,1)]}]

gs_elasticnet = GridSearchCV(ElasticNet(),
                      param_grid=params,
                      scoring='neg_mean_squared_error',
                      cv=5)

gs_elasticnet.fit(x,y)

/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.076e+00, tolerance: 6.345e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.674e+00, tolerance: 7.052e-03 Linear regression models with null weight for the l1 regularization term are more efficien

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid=[{'alpha': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                    14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
                                    25, 26, 27, 28, 29, 30, ...],
                          'l1_ratio': [0]}],
             scoring='neg_mean_squared_error')

In [115]:
gs_elasticnet.best_params_

{'alpha': 25, 'l1_ratio': 0}

### Based on the best set of hyperparameters for each method, find the 10-fold cross-validated prediction error and (on average) the number of variables selected in each case.

In [118]:
def Run_KFold(model, x, y):

    kf = KFold(n_splits = 10, shuffle=True) # set KFOLD
    mse_list = [] # initialize MSE list

    for idxtrain, idxtest in kf.split(x): # run for each of the splits
        xtrain = x[idxtrain]
        ytrain = y[idxtrain]
        ytest = y[idxtest]
        xtest = x[idxtest]

    fitted_model = model.fit(xtrain, ytrain)
    y_pred = fitted_model.predict(xtest)

    mse_list.append(mse(y, y_pred))


In [132]:
Run_KFold(ElasticNet(alpha=25,l1_ratio=0),x,y)

ValueError: Found input variables with inconsistent numbers of samples: [111, 100]